In [7]:
import random
def get_headers():
    with open('user_agent.txt','r') as f:
        users=f.readlines()
        users=[i.rstrip('\n') for i in users]
    return{'User-Agent':random.choice(users)}
def get_proxies():
    with open('douban_https.txt','r') as f:
        https=f.readlines()
        https=[i.rstrip('\n') for i in https]
    return{'https':random.choice(https)}
print(get_headers())
print(get_proxies())

{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.104 Safari/537.36'}
{'https': '119.102.25.64:9999'}


In [ ]:
import requests
from lxml import etree
import re
import pymongo
import time
from random import randint
from multiprocessing import Pool

client=pymongo.MongoClient('localhost',27017)#连接数据库
douban=client['douban']#创建新数据库
movie_items=douban['movie_items']#创建数据集合
insert_num=0


def get_movie_url(url):#获取每部电影的链接
    html=requests.get(url,headers=get_headers(),proxies=get_proxies())
    ids=re.findall(r'"id":"(\d*)"',html.text)
    movie_hrefs=['https://movie.douban.com/subject/'+i+'/' for i in ids]
    
    #验证码手动输入
    if movie_hrefs==[]:
        print(url)
        input('暂停手动输入')
        get_movie_url(url)
    for movie_href in movie_hrefs:
        get_movie_info(movie_href)
        
def get_movie_info(url):#获取电影详细信息
    html=requests.get(url,headers=get_headers(),proxies=get_proxies())
    response=etree.HTML(html.text)
    
    #验证码手动输入
    if not response.xpath('.//h1/span[@property="v:itemreviewed"]/text()'):
        print(url)
        input('暂停手动输入')
        get_movie_info(url)
        pass
    
    movie_item = {}
    try:
        # 电影链接与ID
        movie_item['url']=url
        movie_item['id']=url.split('/')[-2]
        # 电影名
        movie_item['title'] = response.xpath('.//h1/span[@property="v:itemreviewed"]/text()')[0].strip()
        # 上映时间
        release_date=response.xpath('.//span[@property="v:initialReleaseDate"]/text()')
        movie_item['release_date'] = release_date
        # 导演
        movie_item['director'] = response.xpath('.//a[@rel="v:directedBy"]/text()')[0].strip()
        # 电影主演
        stars= response.xpath('.//a[@rel="v:starring"]/text()')
        movie_item['stars']=stars
        # 电影类别
        genre=response.xpath('.//span[@property="v:genre"]/text()')
        movie_item['genre'] =genre
#         # 电影时长
#         movie_item['time'] = response.xpath('.//span[@property="v:runtime"]/text()')[0].strip()
        # 电影的评分
        movie_item['rating_num'] = response.xpath('.//strong[@class="ll rating_num"]/text()')[0].strip()
        # 评分的人数
        movie_item['vote_num'] = response.xpath('.//span[@property="v:votes"]/text()')[0].strip()
        # 电影1-5星的百分比
        movie_item['rating_stars1'] = response.xpath('.//span[@class="rating_per"]/text()')[4].strip()
        movie_item['rating_stars2'] = response.xpath('.//span[@class="rating_per"]/text()')[3].strip()
        movie_item['rating_stars3'] = response.xpath('.//span[@class="rating_per"]/text()')[2].strip()
        movie_item['rating_stars4'] = response.xpath('.//span[@class="rating_per"]/text()')[1].strip()
        movie_item['rating_stars5'] = response.xpath('.//span[@class="rating_per"]/text()')[0].strip() 
        # 电影的短评数
        movie_item['comment_num'] = response.xpath('.//div[@class="mod-hd"]/h2/span/a/text()')[0].strip()
        global insert_num
        insert_num+=1
        print('第',insert_num,'次插入数据')
        print(movie_item)
        movie_items.insert_one(movie_item)#插入数据
    except IndexError:
        print(url,':发生IndexError')
        pass
    
if __name__=='__main__':
    urls=['https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start={}'.format(x) for x in range(0,500,20)]
    pool = Pool(4)  # 创建进程池
    for url in urls:
        pool.apply_async(get_movie_url(url))
    pool.close()
    pool.join()

https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start=0
暂停手动输入d
https://movie.douban.com/j/new_search_subjects?sort=T&range=0,10&tags=%E7%94%B5%E5%BD%B1&start=0
